In [ ]:
!pip install python-box


In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule
from pytorch_lightning.loggers import WandbLogger
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer,AutoModel,AdamW

import wandb
from datasets import load_dataset
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

In [ ]:
wandb.login()


In [ ]:
wandb_logger = WandbLogger(project="Contradictory_Watson")

In [ ]:
config = {'seed': 2021,
          'root': '../input/contradictory-my-dear-watson/train.csv', 
          'n_splits': 3,
          'epoch': 1,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 1,
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          
          
          'train_loader':{
              'batch_size': 64,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': 64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'model':{
              'name': 'xlm-roberta-base',
              'output_dim': 3
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config = Box(config)

# Data

In [ ]:
df=pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

In [ ]:
class Watson_Dataset(Dataset):
    def __init__(self,df):
        super().__init__()
        self.tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-base')
        self.df=df  
        
    def __len__(self):
        return(len(self.df))
    
    def __getitem__(self,idx):
        a=self.tokenizer(self.df.loc[idx,'premise'],self.df.loc[idx,'hypothesis'],max_length=250,padding='max_length',return_tensors='pt')
        return (a,self.df.loc[idx,'label'])
        
        

In [ ]:
mydata=Watson_Dataset(df)

In [ ]:
mydata[0]

In [ ]:
class ConratdictoryDataModule(LightningDataModule):
    def __init__(self,train_df,val_df):
        super().__init__()
        
        self._train_df = train_df
        
       
        self._val_df = val_df
            
        
        
        
    def __create_dataset(self, train=True):
        return (
            Watson_Dataset(self._train_df)
            if train
            else Watson_Dataset(self._val_df)
        )
    
    def train_dataloader(self):
        dataset = self.__create_dataset(True)
        return DataLoader(dataset,batch_size=8,shuffle=True)

    
   
        
    def val_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset,batch_size=2,shuffle=True)
        
        
        

In [ ]:
train_df,val_df=train_test_split(df,test_size=0.2)

In [ ]:
train_df=train_df.reset_index()
val_df=val_df.reset_index()

# Model

In [ ]:
class XLM_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model=AutoModel.from_pretrained('xlm-roberta-base').eval()
        
            
            
    def forward(self,input_ids,attention_mask,labels=None):
        
        out=self.model(input_ids=input_ids,attention_mask=attention_mask)
        
        return out
    
    def training_step(self,batch,batch_idx):
        x=batch[0]
        
        y=batch[1]  
        input_ids=x['input_ids'].squeeze(1)
        
        attention_mask=x['attention_mask'].squeeze(1)
        labels=y
        loss=self.model(input_ids=input_ids,attention_mask=attention_mask)
        #self.log('train_loss',loss,prog_bar=True,logger=True)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x=batch[0]
        y=batch[1]   
        input_ids=x['input_ids'].squeeze(1)
        attention_mask=x['attention_mask'].squeeze(1)
        labels=y
        loss=self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
        #self.log('val_loss',loss,prog_bar=True,logger=True)
        return loss
    
    
    def configure_optimizers(self):
        return AdamW(self.parameters(),lr=0.1)
        
        
        

# Trainer

In [ ]:
lr_monitor = callbacks.LearningRateMonitor()
loss_checkpoint = callbacks.ModelCheckpoint(
filename="best_loss",
monitor="val_loss",
save_top_k=1,
mode="min",
save_last=False,
)


In [ ]:
trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[lr_monitor, loss_checkpoint],

    default_root_dir="/checkpoints",

    max_epochs=5,

    )

In [ ]:
mymodel=XLM_Model()

In [ ]:
py_lightning_data=ConratdictoryDataModule(train_df,val_df)

In [ ]:
trainer.fit(mymodel,py_lightning_data)

In [ ]:
sample_dataloader = ConratdictoryDataModule(train_df, val_df).val_dataloader()
batch= iter(sample_dataloader).next()

In [ ]:
x=batch[0]
y=batch[0]

In [ ]:
input_ids=x['input_ids'].squeeze(1)

In [ ]:
l=mymodel(x['input_ids'].squeeze(1),x['attention_mask'].squeeze(1))

In [ ]:
l.last_hidden_state.shape

In [ ]:
l=mymodel.model(input_ids,attention_mask,m)

In [ ]:
attention_mask=x['attention_mask'].squeeze(1)

In [ ]:
attention_mask.shape